# Project: Disaster Response Pipelines

Margaret Maynard-Reid, 12/13/2020



# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import re
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier

# for 6. Improve your model
from sklearn.model_selection import GridSearchCV 

import numpy as np

# 8. Further improve model
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import SGDClassifier

# Export your model as a pickle file
import pickle

### Load data

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
# engine.table_names() # get a list of table names
df = pd.read_sql_table("message", engine)
df.head(1) # take a look at the dataframe

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message 
X.head(1) # take a look X 

0    Weather update - a cold front from Cuba that c...
Name: message, dtype: object

In [5]:
X.shape

(26028,)

In [6]:
y = df.iloc[:, 4:]
y.head(1) # take a look at y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y.shape

(26028, 35)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])  

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred= pipeline.predict(X_test)

In [12]:
print(classification_report(y_test, y_pred, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.93      0.87      4906
               request       0.83      0.36      0.50      1116
                 offer       0.00      0.00      0.00        32
           aid_related       0.73      0.52      0.61      2653
          medical_help       0.54      0.09      0.15       510
      medical_products       0.65      0.11      0.19       325
     search_and_rescue       0.64      0.10      0.17       184
              security       0.43      0.03      0.05       114
              military       0.60      0.07      0.13       205
                 water       0.87      0.18      0.30       428
                  food       0.83      0.29      0.43       738
               shelter       0.81      0.25      0.38       559
              clothing       0.80      0.04      0.07       109
                 money       0.60      0.02      0.04       148
        missing_people       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f374c2d21e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [13]:
# grid search parameters
parameters = {
        'clf__estimator__max_depth': [2, 5, None],
        'clf__estimator__n_estimators': [10, 20]
    }

# cross validation
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__max_depth': [2, 5, None], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred_cv = cv.predict(X_test)
print(classification_report(y_test, y_pred_cv, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      4993
               request       0.86      0.39      0.54      1107
                 offer       0.00      0.00      0.00        26
           aid_related       0.77      0.55      0.64      2734
          medical_help       0.64      0.05      0.09       536
      medical_products       0.75      0.07      0.13       341
     search_and_rescue       0.50      0.02      0.04       186
              security       1.00      0.02      0.03       132
              military       0.58      0.09      0.15       216
                 water       0.85      0.34      0.49       405
                  food       0.85      0.31      0.46       727
               shelter       0.85      0.20      0.32       612
              clothing       0.89      0.07      0.13       111
                 money       0.75      0.02      0.04       143
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:

better_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SGDClassifier(max_iter=5, tol=None)))])
better_pipeline.fit(X_train, y_train)
y_pred_better = better_pipeline.predict(X_test)
print(classification_report(y_test, y_pred_better, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      4906
               request       0.81      0.61      0.70      1116
                 offer       0.00      0.00      0.00        32
           aid_related       0.75      0.68      0.71      2653
          medical_help       0.66      0.16      0.25       510
      medical_products       0.73      0.22      0.34       325
     search_and_rescue       0.71      0.14      0.23       184
              security       0.00      0.00      0.00       114
              military       0.75      0.10      0.18       205
                 water       0.79      0.59      0.67       428
                  food       0.81      0.74      0.78       738
               shelter       0.80      0.51      0.62       559
              clothing       0.72      0.36      0.48       109
                 money       0.80      0.05      0.10       148
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [16]:
# Save the model to file in the current working directory
pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(better_pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.